В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [68]:
import numpy as np
import math
import scipy.stats
import pandas as pd
from sklearn import model_selection, datasets, linear_model, ensemble, metrics
from statsmodels.stats.weightstats import *

In [109]:
g_a = np.zeros(34)
g_b = np.zeros(16)
for i in range (10):
        g_a[i] = 1
for i in range (4):
        g_b[i] = 1
sum(g_a)

10.0

In [86]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [8]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(g_a, g_b))

95% confidence interval for a difference between proportions: [-0.217558, 0.305793]


In [112]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(g_a, g_b),'greater'))

p-value: 0.372930


Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки:
Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

логистическая регрессия по признакам 
X
1
,
X
2
,
X
3;
логистическая регрессия по признакам 
X
4
,
X
5
,
X
6.
Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 
5.5
×
1
0
−
8
5.5×10 
−8
 , нужно ввести 8).

In [72]:
data = pd.read_csv('banknotes.txt', sep = '\t')

In [27]:
t = data['real']
d = data.drop(columns=['real'])

In [165]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(d, t, random_state=1)

In [160]:
lr_1 = linear_model.LogisticRegression(random_state=1)
lr_2 = linear_model.LogisticRegression(random_state=1)

lr_1.fit(train_data.iloc[:, 0:3], train_labels)
lr_2.fit(train_data.iloc[:, 3:6], train_labels)

y_1 = lr_1.predict(test_data.iloc[:, 0:3])
y_2 = lr_2.predict(test_data.iloc[:, 3:6])

In [161]:
er_1 = metrics.accuracy_score(test_labels, y_1)
er_2 = metrics.accuracy_score(test_labels, y_2)
print (er_1, er_2)

0.8 0.98


In [168]:
print (y_1)
print (y_2)
print(np.array(test_labels))

[1 1 1 0 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0
 0 0 0 1 0 0 1 1 0 0 0 0 0]
[1 1 1 0 0 0 1 1 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 1 1 0 1 1 0 0 0 0 1 0
 1 0 0 0 0 0 1 1 1 0 1 0 0]
[1 1 1 0 0 0 1 1 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 1 1 0 1 1 0 0 0 0 1 0
 1 1 0 0 0 0 1 1 1 0 1 0 0]


In [156]:
# независимые
te = (test_labels!=y_1)*1
te2 = (test_labels!=y_2)*1
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(te, te2)))
print("95%% confidence interval for a difference between proportions: [%f, %f]" % proportions_diff_confint_ind(test_labels!=y_1, test_labels!=y_2))


p-value: 0.004022
95% confidence interval for a difference between proportions: [0.062533, 0.297467]


In [177]:
# зависимые
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(test_labels!=y_1, test_labels!=y_2)))
print("95%% confidence interval for a difference between proportions: [%f, %f]" \
      % proportions_diff_confint_rel(test_labels!=y_1, test_labels!=y_2))

p-value: 0.000923
95% confidence interval for a difference between proportions: [0.073511, 0.286489]


В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

In [135]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" \
      % proportions_diff_confint_rel(test_labels!=y_1, test_labels!=y_2))

95% confidence interval for a difference between proportions: [0.073511, 0.286489]


In [169]:
DescrStatsW(abs(te) - abs(te2)).tconfint_mean()

(0.06970662950654867, 0.29029337049345133)

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки

In [173]:
n=100
xsr = 525
mu = 541.4
sig = 100
z = (xsr-mu)/(sig/(math.sqrt(n)))
#2*(1-scipy.stats.norm.cdf(abs(z)))
#2*(1 - scipy.stats.norm.cdf(z))
scipy.stats.norm.cdf(z)

0.05050258347410395

Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [174]:
n=100
xsr = 525
mu = 541.5
sig = 100
z = (xsr-mu)/(sig/(math.sqrt(n)))
#2*(1-scipy.stats.norm.cdf(abs(z)))
2*(1 - scipy.stats.norm.cdf(z))
scipy.stats.norm.cdf(z)

0.0494714680336481